# Advanced Operations for LLM Input / Output with LangChain

This notebook covers the following operations:

- LLM Cost Monitoring
- Caching
- Streaming

## Install OpenAI, HuggingFace and LangChain dependencies

In [ ]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

## Enter API Tokens

#### Enter your Open AI Key here

You can get the key from [here](https://platform.openai.com/api-keys) after creating an account or signing in

In [2]:
from getpass import getpass

OPENAI_KEY = getpass('Please enter your Open AI API Key here: ')

Please enter your Open AI API Key here: ··········


## Setup necessary system environment variables

In [3]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Chat Models and LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not implement or build its own LLMs. It provides a standard API for interacting with almost every LLM out there.

There are lots of LLM providers (OpenAI, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

## Accessing Commercial LLMs like ChatGPT

In [5]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Tracking LLM Costs

Typically LLMs like ChatGPT charge you based on the number of tokens per request and response. You can track your token usage for specific calls. It is currently only implemented for the OpenAI API.

In [6]:
from langchain_community.callbacks import get_openai_callback

prompt = """Explain Generative AI in one line"""

with get_openai_callback() as cb:
    response = chatgpt.invoke(prompt)
    print(response.content)
    print(cb)

Generative AI refers to algorithms that can create new content, such as text, images, or music, by learning patterns from existing data.
Tokens Used: 42
	Prompt Tokens: 14
	Completion Tokens: 28
Successful Requests: 1
Total Cost (USD): $1.89e-05


In [7]:
cb.total_tokens

42

In [8]:
cb.prompt_tokens, cb.completion_tokens

(14, 28)

In [9]:
cb.total_cost

1.89e-05

## Caching in LangChain

LangChain includes an optional caching layer for language model APIs (LLMs). This caching feature is beneficial for two main reasons:

1. **Cost Efficiency:** By caching responses, you reduce the number of API calls made to LLM providers, especially helpful if you are frequently requesting the same completions. This can significantly lower costs.

2. **Performance Improvement:** Caching can enhance your application's speed by decreasing the need for repeated API calls to the LLM provider, making interactions quicker and more efficient.


#### InMemoryCache

In [10]:
%%time
# integrations with other caching tools:
# https://api.python.langchain.com/en/latest/community_api_reference.html#module-langchain_community.cache
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer

from langchain_core.prompts import ChatPromptTemplate

prompt = """Explain to me what is mortgage"""

chat_template = ChatPromptTemplate.from_template(prompt)

chatgpt.invoke(chat_template.format())

CPU times: user 184 ms, sys: 32.8 ms, total: 217 ms
Wall time: 8.96 s


AIMessage(content='A mortgage is a type of loan specifically used to purchase real estate, such as a home or property. In a mortgage agreement, the borrower (the person buying the property) receives funds from a lender (usually a bank or financial institution) to buy the property. In return, the borrower agrees to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years.\n\nKey components of a mortgage include:\n\n1. **Principal**: The original amount of money borrowed to purchase the property.\n\n2. **Interest**: The cost of borrowing the money, expressed as a percentage of the principal. This is how lenders make a profit on the loan.\n\n3. **Down Payment**: A portion of the purchase price that the borrower pays upfront. This is usually expressed as a percentage of the total price and can vary based on the type of mortgage and lender requirements.\n\n4. **Term**: The length of time over which the loan must be repaid. Common terms are 15, 20, or 30 years.

In [11]:
%%time
# The second time it is, so it goes faster
chatgpt.invoke(chat_template.format())

CPU times: user 596 µs, sys: 0 ns, total: 596 µs
Wall time: 664 µs


AIMessage(content='A mortgage is a type of loan specifically used to purchase real estate, such as a home or property. In a mortgage agreement, the borrower (the person buying the property) receives funds from a lender (usually a bank or financial institution) to buy the property. In return, the borrower agrees to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years.\n\nKey components of a mortgage include:\n\n1. **Principal**: The original amount of money borrowed to purchase the property.\n\n2. **Interest**: The cost of borrowing the money, expressed as a percentage of the principal. This is how lenders make a profit on the loan.\n\n3. **Down Payment**: A portion of the purchase price that the borrower pays upfront. This is usually expressed as a percentage of the total price and can vary based on the type of mortgage and lender requirements.\n\n4. **Term**: The length of time over which the loan must be repaid. Common terms are 15, 20, or 30 years.

#### SQLite Cache

In [12]:
# just to remove cache if it already exists, ignore if you get an error message below, that is normal (when cache doesnt exist)
!rm langchain.db

rm: cannot remove 'langchain.db': No such file or directory


In [14]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="langchain.db"))

In [15]:
%%time

# The first time, it is not yet in cache, so it should take longer
prompt = """Explain to me what is fractional real estate"""

chat_template = ChatPromptTemplate.from_template(prompt)

chatgpt.invoke(chat_template.format())

CPU times: user 27.4 ms, sys: 10.2 ms, total: 37.7 ms
Wall time: 4.33 s


AIMessage(content='Fractional real estate refers to a model of property ownership where multiple individuals or entities share ownership of a single property. This arrangement allows investors to purchase a fraction or percentage of a property rather than the entire asset, making real estate investment more accessible and affordable.\n\nHere are some key features of fractional real estate:\n\n1. **Shared Ownership**: Multiple owners hold a share of the property, which can range from a small percentage to a larger portion, depending on the investment structure.\n\n2. **Cost-Effective**: By sharing the cost of purchasing and maintaining a property, investors can enter the real estate market with a lower financial commitment.\n\n3. **Usage Rights**: Owners typically have specific rights to use the property, which may be allocated based on the percentage of ownership or through a predetermined schedule.\n\n4. **Management**: Fractional ownership often involves a management company that han

In [16]:
%%time
# The second time it is, so it goes faster
chatgpt.invoke(chat_template.format())

CPU times: user 42.3 ms, sys: 41.3 ms, total: 83.6 ms
Wall time: 91.2 ms


AIMessage(content='Fractional real estate refers to a model of property ownership where multiple individuals or entities share ownership of a single property. This arrangement allows investors to purchase a fraction or percentage of a property rather than the entire asset, making real estate investment more accessible and affordable.\n\nHere are some key features of fractional real estate:\n\n1. **Shared Ownership**: Multiple owners hold a share of the property, which can range from a small percentage to a larger portion, depending on the investment structure.\n\n2. **Cost-Effective**: By sharing the cost of purchasing and maintaining a property, investors can enter the real estate market with a lower financial commitment.\n\n3. **Usage Rights**: Owners typically have specific rights to use the property, which may be allocated based on the percentage of ownership or through a predetermined schedule.\n\n4. **Management**: Fractional ownership often involves a management company that han

## Streaming in LLMs

All language model interfaces (LLMs) in LangChain implement the `Runnable` interface, which provides default methods such as `ainvoke`, `batch`, `abatch`, `stream`, and `astream`. This setup equips all LLMs with basic streaming capabilities.

### Streaming Defaults:

- **Synchronous Streaming:** By default, streaming operations return an `Iterator` that yields a single value, the final result from the LLM provider.
- **Asynchronous Streaming:** Similarly, async streaming defaults to returning an `AsyncIterator` with the final result.

### Limitations:

- These default implementations do not support token-by-token streaming. For such detailed streaming, the LLM provider must offer native support. However, the default setup ensures that your code expecting an iterator of tokens will function correctly within these constraints.


In [17]:
prompt = """Explain to me what is mortgage in detail with pros and cons"""
chat_template = ChatPromptTemplate.from_template(prompt)

for chunk in chatgpt.stream(chat_template.format()):
    print(chunk.content)


A
 mortgage
 is
 a
 type
 of
 loan
 specifically
 used
 to
 purchase
 real
 estate
,
 where
 the
 property
 itself
 serves
 as
 collateral
 for
 the
 loan
.
 When
 a
 borrower
 takes
 out
 a
 mortgage
,
 they
 agree
 to
 repay
 the
 loan
 amount
,
 plus
 interest
,
 over
 a
 specified
 period
,
 typically
 
15
 to
 
30
 years
.
 Mort
gages
 are
 commonly
 used
 by
 individuals
 and
 families
 to
 buy
 homes
,
 but
 they
 can
 also
 be
 used
 for
 investment
 properties
 or
 commercial
 real
 estate
.


###
 Key
 Components
 of
 a
 Mortgage
:


1
.
 **
Principal
**
:
 The
 original
 loan
 amount
 borrowed
 from
 the
 lender
.

2
.
 **
Interest
**
:
 The
 cost
 of
 borrowing
 the
 principal
,
 expressed
 as
 a
 percentage
.
 This
 can
 be
 fixed
 (
the
 same
 throughout
 the
 loan
 term
)
 or
 variable
 (
can
 change
 at
 specified
 times
).

3
.
 **
Term
**
:
 The
 length
 of
 time
 over
 which
 the
 loan
 must
 be
 rep
aid
,
 commonly
 
15
,
 
20
,
 or
 
30
 years
.

4
.
 **
Down
 Pay

In [18]:
prompt = """Explain to me what is mortgage in detail with pros and cons"""
chat_template = ChatPromptTemplate.from_template(prompt)

response = []
for chunk in chatgpt.stream(chat_template.format()):
    print(chunk.content, end="")
    response.append(chunk.content)

A mortgage is a type of loan specifically used to purchase real estate, where the property itself serves as collateral for the loan. When a borrower takes out a mortgage, they agree to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years. Mortgages are commonly used by individuals and families to buy homes, but they can also be used for investment properties or commercial real estate.

### Key Components of a Mortgage:

1. **Principal**: The original loan amount borrowed from the lender.
2. **Interest**: The cost of borrowing the principal, expressed as a percentage of the loan amount. This can be fixed (stays the same throughout the loan term) or variable (can change at specified times).
3. **Term**: The length of time over which the loan must be repaid, commonly 15, 20, or 30 years.
4. **Down Payment**: The initial amount paid upfront by the borrower, usually expressed as a percentage of the property’s purchase price. A larger down payment can reduc

In [19]:
response[:10]

['',
 'A',
 ' mortgage',
 ' is',
 ' a',
 ' type',
 ' of',
 ' loan',
 ' specifically',
 ' used']

In [20]:
print(''.join(response))

A mortgage is a type of loan specifically used to purchase real estate, where the property itself serves as collateral for the loan. When a borrower takes out a mortgage, they agree to repay the loan amount, plus interest, over a specified period, typically 15 to 30 years. Mortgages are commonly used by individuals and families to buy homes, but they can also be used for investment properties or commercial real estate.

### Key Components of a Mortgage:

1. **Principal**: The original loan amount borrowed from the lender.
2. **Interest**: The cost of borrowing the principal, expressed as a percentage of the loan amount. This can be fixed (stays the same throughout the loan term) or variable (can change at specified times).
3. **Term**: The length of time over which the loan must be repaid, commonly 15, 20, or 30 years.
4. **Down Payment**: The initial amount paid upfront by the borrower, usually expressed as a percentage of the property’s purchase price. A larger down payment can reduc